In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/open-problems-multimodal/sample_submission.csv
/kaggle/input/open-problems-multimodal/train_cite_targets.h5
/kaggle/input/open-problems-multimodal/metadata_cite_day_2_donor_27678.csv
/kaggle/input/open-problems-multimodal/test_multi_inputs.h5
/kaggle/input/open-problems-multimodal/evaluation_ids.csv
/kaggle/input/open-problems-multimodal/train_cite_inputs.h5
/kaggle/input/open-problems-multimodal/train_multi_targets.h5
/kaggle/input/open-problems-multimodal/train_multi_inputs.h5
/kaggle/input/open-problems-multimodal/metadata.csv
/kaggle/input/open-problems-multimodal/test_cite_inputs_day_2_donor_27678.h5
/kaggle/input/open-problems-multimodal/test_cite_inputs.h5
/kaggle/input/open-problems-msci-multiome-sparse-matrices/train_multi_targets_sparse.npz
/kaggle/input/open-problems-msci-multiome-sparse-matrices/test_multi_inputs_sparse.npz
/kaggle/input/open-problems-msci-multiome-sparse-matrices/train_multiome_input_sparse.npz


In [2]:
# 导入库
import pandas as pd
import numpy as np
from tqdm import tqdm 
import gc
import os
import sys
from scipy import sparse
import torch
import torch.nn.functional as F
from torch import Tensor
!pip install tables

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 4.5 MB/s eta 0:00:00


In [3]:
# cite 数据准备
cite_x = pd.read_hdf('../input/open-problems-multimodal/train_cite_inputs.h5').values
sparse.save_npz('./train_cite_inputs_sparse',sparse.csr_matrix(cite_x))
del cite_x
gc.collect()
cite_y = pd.read_hdf('../input/open-problems-multimodal/train_cite_targets.h5').values
sparse.save_npz('./train_cite_targets_sparse',sparse.csr_matrix(cite_y))
del cite_y
gc.collect()
cite_x_test = pd.read_hdf('../input/open-problems-multimodal/test_cite_inputs.h5').values
sparse.save_npz('./test_cite_inputs_sparse',sparse.csr_matrix(cite_x_test))
del cite_x_test
gc.collect()

58

In [4]:
# multi std and mean
multi_y = sparse.load_npz('../input/open-problems-msci-multiome-sparse-matrices/train_multi_targets_sparse.npz')
y_mean = np.array(multi_y.mean(axis=1)).reshape(-1)
multi_y_c = multi_y.copy()
multi_y_c.data**=2
y_std = np.sqrt(np.array(multi_y_c.mean(axis=1)).reshape(-1)-y_mean)
np.save(os.path.join('./','multi_y_mean'), y_mean, allow_pickle=True)
np.save(os.path.join('./','multi_y_std'), y_std, allow_pickle=True)
index = 456
check_1 = multi_y[index]
check_1_mstd = (check_1.toarray()-y_mean[index])/y_std[index]
print(np.mean(check_1_mstd))
print(np.std(check_1_mstd))
del multi_y,multi_y_c,check_1,check_1_mstd,index,y_std,y_mean
gc.collect()

-9.382806e-08
0.9978744


40

In [5]:
# submit multi
multi_target_col = pd.read_hdf('../input/open-problems-multimodal/train_multi_targets.h5',start=0,stop=1).columns
eval_ids = pd.read_csv('../input/open-problems-multimodal/evaluation_ids.csv', index_col='row_id')
# Convert the string columns to more efficient categorical types
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())
cell_id_set = set(eval_ids.cell_id)
eval_ids_col = eval_ids.groupby('cell_id')['gene_id'].apply(lambda x:','.join(x))
eval_ids_index = eval_ids.cell_id
del eval_ids
gc.collect()

0

In [6]:
def ret_index(x):
    col_lst = x.split(',')
    return np.where(multi_target_col.isin(col_lst))[0]
start = 0
chunksize = 1000
total_rows = 0
row_lst = []
row_name_lst = []
col_lst = []
while True:
    print(start)
    multi_test_x = None # Free the memory if necessary
    gc.collect()
    multi_test_x = pd.read_hdf('../input/open-problems-multimodal/test_multi_inputs.h5', start=start, stop=start+chunksize)
    rows_read = len(multi_test_x)
    row_mask = multi_test_x.index.isin(cell_id_set)
    row_lst+=row_mask.tolist()
    multi_test_x_index = multi_test_x.loc[row_mask].index
    row_name_lst+=multi_test_x_index.tolist()
    if len(multi_test_x_index) > 0:
        col_lst.append(np.stack(eval_ids_col[multi_test_x_index].apply(lambda x:ret_index(x)).values))
    if rows_read < chunksize: 
        break
    start += chunksize
row_lst = np.array(row_lst)
row_name_lst = np.array(row_name_lst)
col_lst = np.concatenate(col_lst)
col_name_lst = np.array(multi_target_col)
assert col_lst.shape[0] == np.sum(row_lst)
assert col_lst.shape[0] == len(row_name_lst)
np.save(os.path.join('./','multi_test_row'), row_lst, allow_pickle=True)
np.save(os.path.join('./','multi_test_row_name'), row_name_lst, allow_pickle=True)
np.save(os.path.join('./','multi_test_col'), col_lst, allow_pickle=True)
np.save(os.path.join('./','multi_test_col_name'), col_name_lst, allow_pickle=True)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
